In [3]:
import numpy as np
import pandas as pd

rating_df = pd.read_csv("Data/rating.csv")
anime_df = pd.read_csv("Data/anime.csv")
print("read")

read


In [78]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [79]:
anime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


Lets remove columns that we are not going to use right now. Maybe in the future we can use them to better predict what to recommend for our costumers.

In [80]:
#We don't need 'rating' column, we are going to use the ratings from the ratings dataframe.
#The number of members, the type and the numer of the episodes in the anime maybe be a good estimador,
#but now we are going to focus only on the anime's genre.

animes_with_genres_df = anime_df.copy()

animes_with_genres_df.drop(['rating', 'members', 'type', 'episodes'], axis=1, inplace=True)
animes_with_genres_df.head()

,anime_id,name,genre
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural"
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S..."
3,9253,Steins;Gate,"Sci-Fi, Thriller"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S..."


In [81]:
animes_with_genres_df.isna().sum()

anime_id     0
name         0
genre       62
dtype: int64

Temos alguns valores desconhecidos (NaN) no dataset, nas colunas genre, type e rating, aparentemente são poucos, então podemos lidar mais tranquilamente com eles, talvez mesmo remove-los do dataset.

In [82]:
animes_with_genres_df.dropna(inplace=True)
animes_with_genres_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12232 entries, 0 to 12293
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   anime_id  12232 non-null  int64 
 1   name      12232 non-null  object
 2   genre     12232 non-null  object
dtypes: int64(1), object(2)
memory usage: 382.2+ KB


A coluna "genre" possui valores em string com uma lista das categorias na qual aquele anime se encaixa. Dados assim não ajudam nossos algoritmos de Machine Learning, então a primeira etapa será criar uma coluna no dataset para cada gênero de anime, e colocar um valor de 1 para se o anime pertencer àquele gênero, e 0 para se ele não pertencer.

In [83]:
for index, row in animes_with_genres_df.iterrows():
    for genre in row.genre.split(", "):
        animes_with_genres_df.at[index, genre] = 1
        
animes_with_genres_df.fillna(0, inplace=True)
animes_with_genres_df.head()

,anime_id,name,genre,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,...,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai,Hentai,Yaoi,Yuri
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,Steins;Gate,"Sci-Fi, Thriller",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Agora temos uma coluna para representar cada gênero possível do anime, e o valor 1 para se o anime pertencer àquele gênero e 0 para se ele não pertencer.

In [84]:
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [85]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


Agora, vamos criar um content-based recommendation system. O foco dele será recomendar animes novos para um possível consumidor, baseado em notas que o cliente deu para outros títulos. Levaremos em conta os gêneros dos animes para recomendar similares a ele.

In [86]:
animes_with_genres_df.tail()

,anime_id,name,genre,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,...,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai,Hentai,Yaoi,Yuri
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12290,5543,Under World,Hentai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12291,5621,Violence Gekiga David no Hoshi,Hentai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12293,26081,Yasuji no Pornorama: Yacchimae!!,Hentai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [87]:
#Vamos criar um consumidor ficticio, com alguns animes na lista dele e algumas notas.
user_test = [
    {'name': 'Kimi no Na wa.', 'rating': 10},
    {'name': 'Fullmetal Alchemist: Brotherhood', 'rating': 10},
    {'name': 'Naruto', 'rating': 8},
    {'name': 'Under World', 'rating': 3},
    {'name': 'Yasuji no Pornorama: Yacchimae!!', 'rating': 4},
]

user_test_df = pd.DataFrame(user_test)
user_test_df.head()

,name,rating
0,Kimi no Na wa.,10
1,Fullmetal Alchemist: Brotherhood,10
2,Naruto,8
3,Under World,3
4,Yasuji no Pornorama: Yacchimae!!,4


In [88]:
# Vamos unir a lista com os dados do titulo que o usuario assistiu com o respectivo titulo no dataframe
#dos animes.
user_test_df = pd.merge(animes_with_genres_df, user_test_df, on ="name")

# Vamos separar os dados agora para um dataframe que contem apenas o id do anime, o nome e a 
#nota que o cliente deu
# E vamos criar uma lista com os mesmos dados (exceto rating) e os restante dos dados
user_test_animes = user_test_df.drop(['rating'], axis=1)
user_test_df = user_test_df[['anime_id', 'name', 'rating']]

print(user_test_animes)
print(user_test_df)

   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2        20                            Naruto   
3      5543                       Under World   
4     26081  Yasuji no Pornorama: Yacchimae!!   

                                               genre  Drama  Romance  School  \
0               Drama, Romance, School, Supernatural    1.0      1.0     1.0   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...    1.0      0.0     0.0   
2  Action, Comedy, Martial Arts, Shounen, Super P...    0.0      0.0     0.0   
3                                             Hentai    0.0      0.0     0.0   
4                                             Hentai    0.0      0.0     0.0   

   Supernatural  Action  Adventure  Fantasy  ...  Shounen Ai  Game  Dementia  \
0           1.0     0.0        0.0      0.0  ...         0.0   0.0       0.0   
1           0.0     1.0        1.0      1.0  ...         0.0   0

In [89]:
# Agora vamos pegar apenas os gêneros da lista do cliente, mas antes vamos reiniciar os index para
#evitar problemas futuros
user_test_animes = user_test_animes.reset_index(drop=True)
user_test_genres = user_test_animes.drop(['anime_id', 'name', 'genre'], axis=1)
user_test_genres

,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,Magic,Military,Shounen,...,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai,Hentai,Yaoi,Yuri
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


Agora, iremos multiplicar a nota dada pelo cliente pelos gêneros dos títulos, assim transformaremos cada gênero em um peso para a avaliação final. Como é um produto escalar, podemos usar a função **dot** do pandas

In [90]:
user_profile = user_test_genres.transpose().dot(user_test_df['rating'])
user_profile

Drama            20.0
Romance          10.0
School           10.0
Supernatural     10.0
Action           18.0
Adventure        10.0
Fantasy          10.0
Magic            10.0
Military         10.0
Shounen          18.0
Comedy            8.0
Historical        0.0
Parody            0.0
Samurai           0.0
Sci-Fi            0.0
Thriller          0.0
Sports            0.0
Super Power       8.0
Space             0.0
Slice of Life     0.0
Mecha             0.0
Music             0.0
Mystery           0.0
Seinen            0.0
Martial Arts      8.0
Vampire           0.0
Shoujo            0.0
Horror            0.0
Police            0.0
Psychological     0.0
Demons            0.0
Ecchi             0.0
Josei             0.0
Shounen Ai        0.0
Game              0.0
Dementia          0.0
Harem             0.0
Cars              0.0
Kids              0.0
Shoujo Ai         0.0
Hentai            7.0
Yaoi              0.0
Yuri              0.0
dtype: float64

Esses pesos são conhecidos como "Perfil do usuário", baseado neles podemos recomendar novos títulos para o cliente

Agora vamos pegar a lista de gêneros do dataframe original

In [91]:
genre_table = animes_with_genres_df.set_index(animes_with_genres_df['anime_id'])

genre_table.drop(['anime_id', 'name', 'genre'], axis=1, inplace=True)
genre_table.head()

,Drama,Romance,School,Supernatural,Action,Adventure,Fantasy,Magic,Military,Shounen,...,Shounen Ai,Game,Dementia,Harem,Cars,Kids,Shoujo Ai,Hentai,Yaoi,Yuri
anime_id,,,,,,,,,,,,,,,,,,,,,
32281,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5114,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28977,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9969,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
genre_table.shape

(12232, 43)

Agora vamos pegar a nossa lista de gêneros, nossos pesos e calcular a média ponderada de cada título para recomendar os que mais batem com o gosto do cliente.

In [94]:
recommendations_df = ((genre_table * user_profile).sum(axis=1)) / (user_profile.sum())
recommendations_df.head()

anime_id
32281    0.318471
5114     0.611465
28977    0.280255
9253     0.000000
9969     0.280255
dtype: float64

In [95]:
# Vamos ordená-los em ordem decrescente
recommendations_df = recommendations_df.sort_values(ascending=False)
recommendations_df.head()

anime_id
231      0.713376
4938     0.675159
9135     0.662420
121      0.662420
34055    0.624204
dtype: float64

In [97]:
anime_df.loc[anime_df['anime_id'].isin(recommendations_df.head(10).keys())]

,anime_id,name,genre,type,episodes,rating,members
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
200,121,Fullmetal Alchemist,"Action, Adventure, Comedy, Drama, Fantasy, Mag...",TV,51,8.33,600384
286,4938,Tsubasa: Shunraiki,"Action, Adventure, Drama, Fantasy, Magic, Myst...",OVA,2,8.23,40420
1071,969,Tsubasa Chronicle 2nd Season,"Action, Adventure, Drama, Fantasy, Mystery, Ro...",TV,26,7.70,79166
1558,9135,Fullmetal Alchemist: The Sacred Star of Milos,"Action, Adventure, Comedy, Drama, Fantasy, Mag...",Movie,1,7.50,87944
1845,25157,Trinity Seven,"Action, Comedy, Ecchi, Fantasy, Harem, Magic, ...",TV,12,7.43,208796
3283,6489,Zero no Tsukaima: Princesses no Rondo Picture ...,"Action, Adventure, Comedy, Drama, Ecchi, Fanta...",Special,7,7.04,23532
3316,2832,Ani*Kuri15,"Adventure, Comedy, Drama, Fantasy, Game, Magic...",Special,15,7.02,12926
5917,231,Asagiri no Miko,"Action, Comedy, Drama, Fantasy, Magic, School,...",TV,26,6.31,4721
10924,34055,Berserk (2017),"Action, Adventure, Demons, Drama, Fantasy, Hor...",TV,Unknown,NaN,13463
